In [1]:
import cv2
import numpy as np
import matplotlib.image as mpimg
from mtcnn.mtcnn import MTCNN
from PIL import Image

import os

# MAKING OUR DATASET

In [12]:
camera = cv2.VideoCapture(0)
camera.set(3,480) #width
camera.set(4,640) #height
detector = MTCNN()

print("Enter Id")
id = input()
print("Look the camera and wait ...")

count = 0

while(True):
    _, frame = camera.read()
    faces = detector.detect_faces(frame)
    for face in faces:
        (x,y,w,h) = face['box']
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)   
        count += 1
        cv2.imwrite("data/user." + str(id) + '.' +  str(count) + ".jpg", frame[y:y+h,x:x+w])
    cv2.imshow('video',frame)    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif count >= 10: 
         break
            
print("Done ...")
camera.release()
cv2.destroyAllWindows()

Enter Id
Look the camera and wait ...
Done ...


# TRAINING OUR MODEL

In [17]:
model = cv2.face.LBPHFaceRecognizer_create()

def faces_and_ids():
    paths = [os.path.join('data',f) for f in os.listdir('data')]     
    faces = []
    ids = []
    for path in paths:
        img = Image.open(path).convert('L') # grayscale
        img = np.array(img,'uint8')
        id = int(os.path.split(path)[-1].split(".")[1])
        faces.append(img)
        ids.append(id)
    return faces,ids


print("Training ...")
faces, ids = faces_and_ids()
model.train(faces, np.array(ids))
model.write('trainer.yml') 
print("Done ...")




AttributeError: module 'cv2' has no attribute 'face'

# DETECTION

In [65]:
model = cv2.face.LBPHFaceRecognizer_create()
model.read('trainer.yml')
detector = MTCNN()


font = cv2.FONT_HERSHEY_SIMPLEX

id = 0
names = ['None', 'Nandini', 'unknown', 'unknown'] 

In [66]:
camera = cv2.VideoCapture(0)
camera.set(3, 480) #width
camera.set(4, 640) #height

while True:
    _, frame = camera.read()
    faces = detector.detect_faces(frame)
    gray = cv2.cvtColor(frame ,cv2.COLOR_BGR2GRAY)
    for face in faces:
        (x,y,w,h) = face['box']
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        id, confidence = model.predict(gray[y:y+h,x:x+w])
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(
                    frame, 
                    str(id), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
        cv2.putText(
                    frame, 
                    str(confidence), 
                    (x+5,y+h-5), 
                    font, 
                    1, 
                    (255,255,0), 
                    1
                   )  
    
    cv2.imshow('output',frame) 
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break

camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 157ms/step


1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 58ms/step


1/1 [==============================] - 0s 128ms/step
